In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Friday 10/28, 11:59 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset---i.e. it's kind of a mess. The full dataset contains a giant `data` table of sensor readings that is many billions of readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

The schema for the database is shown below. Sometimes people think that if the data is a nice schema, then it's ready to go! We'll see about that.

**Note:** Each line represents a relationship between the two fields. The side of the line diverging to three lines / arrows suggests the "many" side of the relationship, while the side of the line converging to one arrow suggests the "one" side of the relationship.

<img src="data/schema.png">

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests. Free-response questions will be manually graded.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
1a	| 1
1b  | 1
1c	| 1
1d	| 2
2a	| 3
2b	| 1
3a	| 0
3b	| 1
3c	| 1
3d  | 0
3e  | 2
4a	| 2
4b	| 2
4c	| 3
**Total** | 20

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/template1

## Loading Up the Database
To load the database, run the following cell.

**IMPORTANT NOTE:** In contrary to previous projects, the database does not reload automatically upon running the following loading cell. If you would like to reload the database (e.g. if your database is modified in undesirable ways), **first restart the kernel**, and then run `!psql -h localhost -c 'DROP DATABASE IF EXISTS ucb_buildings'` in another cell **before** running the cell below.

In [4]:
import subprocess
import os
import warnings

call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)

if call.stdout != "1\n":
    print("Loading database...")
    os.system("gunzip -c data/proj3.sql.gz | psql -h localhost -d template1 -f -")
else:
    print("Database was previously loaded.")

Database was previously loaded.


Now, run the following cell to connect to the `ucb_buildings` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

Run the following cell for grading purposes.

In [6]:
!mkdir -p results

<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

Note that the `data` table, in the full database, is billions of rows. What do you notice about the design of the database schema that helps support the large amount of data?

**Hint:** There is no need to examine any data here. What is a technique learned in [lecture 15](https://docs.google.com/presentation/d/1GogwyAylHrJoxer_apIkgnWDqC9LxTQ8/edit)? Define that technique.

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 1
-->

Entity relation, Using the primary and foreign keys to represents a relationship between entities.

<!-- END QUESTION -->



### Question 1b

The diagram claims that `buildings_site_mapping` has a many-to-many relationship with `real_estate_metadata`. Let's validate that. 

Below is an example of `json_agg` being used with a table; you will need to do this in the next two parts.

In [7]:
%%sql
SELECT b.site, json_agg(b) from buildings_site_mapping b GROUP BY b.site LIMIT 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.


site,json_agg
Sutardja Dai Hall,"[{'site': 'Sutardja Dai Hall', 'building': 'SUTARDJA DAI'}]"
Le Conte Hall,"[{'site': 'Le Conte Hall', 'building': 'LE CONTE'}]"
Hildebrand Hall,"[{'site': 'Hildebrand Hall', 'building': 'HILDEBRA ND'}]"
Stephens Hall,"[{'site': 'Stephens Hall', 'building': 'STEPHENS'}]"
2000 Carleton Street,"[{'site': '2000 Carleton Street', 'building': 'CARLETO2 000'}]"


Find the **distinct** values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. Order your final result by building.

**Hint:** You should use a CTE to find the distinct buildings of `buildings_site_mapping` before applying necessary table joins.

In [8]:
%%sql
WITH dist_building AS (SELECT DISTINCT building FROM buildings_site_mapping)
SELECT r.building_name, r.json_agg 
FROM dist_building JOIN (SELECT r.building_name, json_agg(r) from real_estate_metadata r GROUP BY r.building_name HAVING COUNT(*) > 1) as r
ON dist_building.building = r.building_name;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.


building_name,json_agg
ALUMNI HOUSE,"[{'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}, {'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}]"
CAMPBELL,"[{'location': 'BERKELEY', 'building': '1027', 'building_name': 'CAMPBELL', 'address': 'CORE C28A5M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 83373, 'bldg_asf': 50745, 'levels': 7, 'year': '2014', 'owner': 'UC', 'mp_code': 'P', 'book_value': 75256344.0}, {'location': 'LOS ANGELES', 'building': '4294', 'building_name': 'CAMPBELL', 'address': 'PORTOLA P11L8A1Z1A', 'city_name': 'LOS ANGELES', 'county': 'LOS ANGELES', 'category': 'GENERAL', 'osfg': 55401, 'bldg_asf': 32619, 'levels': 5, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 6247192.0}]"
HERTZ,"[{'location': 'DAVIS', 'building': '9454', 'building_name': 'HERTZ', 'address': '7000 EAST 2A4V0E0 LAKE', 'city_name': 'LIVERMOR E', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 10904, 'bldg_asf': 8183, 'levels': 1, 'year': '1976', 'owner': 'UC', 'mp_code': 'P', 'book_value': 0}, {'location': 'BERKELEY', 'building': '1423', 'building_name': 'HERTZ', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 31362, 'bldg_asf': 13518, 'levels': 4, 'year': '1958', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3491181.0}]"
SOUTH,"[{'location': 'DAVIS', 'building': '3275', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 26594, 'bldg_asf': 15323, 'levels': 4, 'year': '1912', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3585399.0}, {'location': 'BERKELEY', 'building': '1484', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 30401, 'bldg_asf': 15473, 'levels': 4, 'year': '1873', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1426174.0}]"
SPROUL,"[{'location': 'RIVERSIDE', 'building': 'P5523', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'RIVERSIDE', 'county': 'RIVERSIDE', 'category': 'GENERAL', 'osfg': 80988, 'bldg_asf': 45532, 'levels': 5, 'year': '1965', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1296147.0}, {'location': 'BERKELEY', 'building': '1210', 'building_name': 'SPROUL', 'address': 'CORE 2C6A0M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 111197, 'bldg_asf': 66680, 'levels': 6, 'year': '1941', 'owner': 'UC', 'mp_code': 'P', 'book_value': 4278819.0}, {'location': 'DAVIS', 'building': '3815', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 53874, 'bldg_asf': 27913, 'levels': 10, 'year': '1963', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1620880.0}]"


In [9]:
%%sql result_1b <<
WITH dist_building AS (SELECT DISTINCT building FROM buildings_site_mapping)
SELECT r.building_name, r.json_agg 
FROM dist_building JOIN (SELECT r.building_name, json_agg(r) from real_estate_metadata r GROUP BY r.building_name HAVING COUNT(*) > 1) as r
ON dist_building.building = r.building_name;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.
Returning data to local variable result_1b


In [10]:
# Do not delete/edit this cell
result_1b.DataFrame().to_csv('results/result_1b.csv', index=False)

In [11]:
grader.check("q1b")

q1b results: All test cases passed!

### Question 1c

Now find examples of many matches in the opposite direction. For each distinct `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. Order your final result by building name.

**Hint:** You should use a CTE to find the distinct building names of `real_estate_metadata` before applying necessary table joins.

In [12]:
%%sql 
WITH dist_restate AS (SELECT DISTINCT building_name FROM real_estate_metadata)
SELECT b.building, b.json_agg 
FROM dist_restate JOIN (SELECT b.building, json_agg(b) from buildings_site_mapping b GROUP BY b.building HAVING COUNT(*) > 1) as b
ON dist_restate.building_name = b.building;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.


building,json_agg
EDWARDS FLD,"[{'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}]"
HAAS STU BLD,"[{'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}, {'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}]"
FAC CLUB,"[{'site': ""Men's Faculty Club"", 'building': 'FAC CLUB'}, {'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}]"
SIMON,"[{'site': 'Simpson Center', 'building': 'SIMON'}, {'site': 'Boalt and Simon Hall', 'building': 'SIMON'}]"


In [13]:
%%sql result_1c <<
WITH dist_restate AS (SELECT DISTINCT building_name FROM real_estate_metadata)
SELECT b.building, b.json_agg 
FROM dist_restate JOIN (SELECT b.building, json_agg(b) from buildings_site_mapping b GROUP BY b.building HAVING COUNT(*) > 1) as b
ON dist_restate.building_name = b.building;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.
Returning data to local variable result_1c


In [14]:
# Do not delete/edit this cell
result_1c.DataFrame().to_csv('results/result_1c.csv', index=False)

In [15]:
grader.check("q1c")

q1c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 1d

Do you see any issues with the schema given? In particular, please address the two questions below:
- Can you uniquely determine the building given the sensor data? Why? (**Hint:** given a row in the `data` table, can you determine a **uniquely** associated row in `real_estate_metadata` table? Your answer should draw insights from 1b.)
- Could `buildings_site_mapping.building` be a valid foreign key pointing to `real_estate_metadata.building_name`? (**Hint:** think about the definition / constraints of a foreign key.)

<!--
BEGIN QUESTION
name: q1d
manual: true
points: 2
-->

In [16]:
%%sql
SELECT * FROM data LIMIT 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.


time,id,value
2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.74
2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.51
2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.27
2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.02
2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.77


We cannot uniquely determine the building given the sensor data, since there could be multiple buildings associated with a given building name.

buildings_site_mapping.building cannot be foreign key pointing to real_estate_metadata.building_name. Forign keys (primary keys) must be unique.

<!-- END QUESTION -->



## Question 2: Looking for Outliers in the Readings
Physical sensors like the ones generating this data are notorious for producing crazy outliers on occasion. In this section we'll do a little data cleaning of the outliers.

All the readings from all different kinds of sensors are mixed together in the `data` table. This hodgepodge of mixed readings is going to require us to do some extra work to look for outliers. Let's get started.

### Question 2a: Outlier Detection

Let's find the outlying values *for each sensor id*. We'll call something an outlier if it is **3 Hampel X84 intervals** away from the median. If 

Specifically, create a view `labeled_data` that contains all of the columns in `data` and adds three additional columns at the far right:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the Median Absolute Deviation (MAD),
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the mad is 0, set this to `false`.

In [17]:
%%sql
CREATE OR REPLACE VIEW median AS
    select id, percentile_disc(0.5) WITHIN GROUP (ORDER BY value) as median
    from data
            group by id;

CREATE OR REPLACE VIEW mad AS
    WITH absdevs AS 
        (SELECT data.id, abs(value - median) as d 
             FROM data Join median on data.id = median.id)
    select id, percentile_disc(0.5) WITHIN GROUP (ORDER BY d) as mad
        from absdevs 
            group by id;


   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
Done.


[]

In [18]:
%%sql result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
SELECT time, data.id, value, median, mad, 
    CASE WHEN value BETWEEN (median-3*1.4826*mad) AND (median+3*1.4826*mad) or mad = 0 THEN false
            ELSE true
        END AS outlier
from data join median on data.id = median.id join mad on mad.id = data.id;
SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2a


In [19]:
# Do not delete/edit this cell
result_2a.DataFrame().to_csv('results/result_2a.csv', index=False)

In [20]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b: Outlier Handling (Winsorization)

In this step we'll define a view `cleaned_data` over all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be the same as the original value.

In [21]:
%%sql result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
select *,
    case WHEN mad = 0 or not outlier THEN value 
        WHEN outlier and value > median+3*1.4826*mad THEN median+3*1.4826*mad
        WHEN outlier and value < median-3*1.4826*mad THEN median-3*1.4826*mad
            END AS clean_value
from labeled_data;
SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2b


In [22]:
# Do not delete/edit this cell
result_2b.DataFrame().to_csv('results/result_2b.csv', index=False)

In [23]:
grader.check("q2b")

q2b results: All test cases passed!

<!-- BEGIN QUESTION -->

## Question 3: Entity Resolution

### Question 3a
There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table. What do you notice about these values? Are there any duplicates?

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 0
-->

By looking at the distinct units of metadata, some units have multiple names although they are the same. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b

Sometimes, entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case. The two columns can have arbitrary names.

<!--
BEGIN QUESTION
name: q3bm
manual: true
points: 0
-->

In [24]:
%%sql result_3b <<
CREATE OR REPLACE VIEW case_ignored as select LOWER(units) as units, count(*) as ignored_count from metadata group by LOWER(units);

with unit_count AS (select units, count(*) as count from metadata group by units) 

select unit_count.count, ignored_count 
    from unit_count join case_ignored 
        on LOWER(unit_count.units) = case_ignored.units;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
34 rows affected.
Returning data to local variable result_3b


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [25]:
# Do not delete/edit this cell
result_3b.DataFrame().to_csv('results/result_3b.csv', index=False)

In [26]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3c

Arguably we shouldn't care about these alternative unit labels, *as long as each sensor class uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns single row with one column of value `true` if the condition in italics above holds, or a single row with one column of value `false` otherwise. Please do not hard code this query - we reserve the right to penalize your score if you do so.

<!--
BEGIN QUESTION
name: q3cm
manual: true
points: 0
-->

In [27]:
%%sql result_3c <<
with counts as (select count(distinct units) as count 
                        from metadata group by class)
SELECT 
    CASE WHEN max(count) > 1 Then false
    else true
    end as bool from counts;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_3c


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [28]:
# Do not delete/edit this cell
result_3c.DataFrame().to_csv('results/result_3c.csv', index=False)

In [29]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d

Moving on, have a look at the `real_estate_metadata` table---starting with the distinct values in the `location` field! What do you notice about these values?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 0
-->

Many names with identical meaning but slightly different representation. Show similarity by having similar characters.

<!-- END QUESTION -->



### Question 3e

It turns out this table was the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We'll just clean up the `location` column for now, and leave you to imagine the effort to do a full cleanup of all columns.

To provide some useful utility functions, we have preloaded Postgres' extension packages for "fuzzy" string matching and trigrams for you. You can use any of the string functions in those packages if you like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of canonical names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), write a SQL query that returns the columns `(building_name, address, location, clean_location)` where `clean_location` contains the best match from `uc_locations.loc_name`. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

In [30]:
%%sql result_3e <<
CREATE OR REPLACE VIEW sim as select distinct real_estate_metadata.location, uc_locations.loc_name as clean_location,
similarity(real_estate_metadata.location, uc_locations.loc_name) as sim
from real_estate_metadata, uc_locations
where similarity(real_estate_metadata.location, uc_locations.loc_name) > .25;

with sim_modified as (select location, clean_location 
from sim where sim not between .3 and .4)
select re.building_name, re.address, re.location, sim.clean_location
from real_estate_metadata re join sim_modified sim on re.location = sim.location;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5276 rows affected.
Returning data to local variable result_3e


In [31]:
# Do not delete/edit this cell
result_3e.DataFrame().sort_values(['clean_location', 'building_name', 'address']).iloc[::10].to_csv('results/result_3e.csv', index=False)

In [32]:
grader.check("q3e")

q3e results: All test cases passed!

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. E.g. to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. E.g. a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- Alternatively, the [EXTRACT](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function is sometimes handy. Note the special `EXTRACT(EPOCH FROM ...)` case. This converts a timestamp into an integer representing the number of seconds since midnight, 1/1/1970 (the dawn of [UNIX time](https://en.wikipedia.org/wiki/Unix_time)!)  You can do normal integer comparisons and arithmetic on the results.
- You will need to use the [lag](https://www.geeksforgeeks.org/postgresql-lag-function/) function as the window function.


Create a view called `gaps` that augments the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

The view should only contain rows where `timediff` is **greater than or equal to 30 minutes**.

In [33]:
%%sql result_4a <<
CREATE OR REPLACE VIEW gaps AS
with tmp as (select date_trunc('minute', time) as time, id, value,
             lag(date_trunc('minute', time), 1) over (partition by id order by time) as lagtime, 
             lag(value, 1) over (partition by id order by time) as lagvalue 
             from data)
select *, time - lagtime as timediff from tmp 
where time - lagtime >= interval '30 minutes';
SELECT * FROM gaps ORDER BY id, time LIMIT 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
10 rows affected.
Returning data to local variable result_4a


In [34]:
# Do not delete/edit this cell
result_4a.DataFrame().to_csv('results/result_4a.csv', index=False)

In [35]:
grader.check("q4a")

q4a results: All test cases passed!

### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function* as we did in lecture 12 (when we manufactured data from a normal distribution). The table-valued function we want here is `generate_series` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `generate_series`, consider the following simple query that generates a table of integers with intervals of size 3 between them.

In [36]:
%%sql
SELECT *
  FROM generate_series(1, 10, 3);

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.


generate_series
1
4
7
10


Now, we can use `generate_series` in a `LATERAL JOIN` query: for each tuple on the left of the `LATERAL` it will produce a series based on the values of that tuple. So for example, we can generate 2 tuples for each tuple of `uc_locations` as follows:

In [37]:
%%sql
SELECT loc_id, loc_name, length(loc_name), newval
  FROM uc_locations u, 
       LATERAL generate_series(length(loc_name), length(loc_name) + 2, 2) AS newval;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
24 rows affected.


loc_id,loc_name,length,newval
1,AG FIELD STAT,13,13
1,AG FIELD STAT,13,15
2,BERKELEY,8,8
2,BERKELEY,8,10
3,DAVIS,5,5
3,DAVIS,5,7
4,IRVINE,6,6
4,IRVINE,6,8
5,LOS ANGELES,11,11
5,LOS ANGELES,11,13


Notice how the 2 values it generates are the length of the `loc_name`, and the length + 2. You might want to play with the query above to make sure you understand the documentation for `generate_series` and `LATERAL`.

Ok, on to your task!

Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than or equal to 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, **with `NULL` as the value**. You probably want to use your `gaps` view as well as `generate_series` to do this!

**Hint:** the lower and upper bounds in generate_series (in pseudocode) should be `(lagtime + 15 minutes, time - 15 minutes)`.

In [38]:
%%sql
with missing as (select g_time, id, null as value
                 from gaps, LATERAL generate_series(lagtime + interval '15 minute', 
                            time - interval '15 minutes', interval '15 minutes') as g_time)
select g_time, id, value from missing limit 50;
select time, id, value from data limit 50

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
50 rows affected.
50 rows affected.


time,id,value
2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.74
2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.51
2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.27
2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.02
2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.77
2018-06-12 21:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.54
2018-06-12 21:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.3
2018-06-12 21:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.04
2018-06-12 21:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.8
2018-06-12 20:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.57


In [39]:
%%sql
with missing as (select g_time, id, null as value
                 from gaps, LATERAL generate_series(lagtime + interval '15 minute', 
                            time - interval '15 minutes', interval '15 minutes') as g_time)
SELECT g_time as time, id, CAST(value AS real) FROM missing
UNION ALL
SELECT time, id, value FROM data
limit 50;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
50 rows affected.


time,id,value
2018-06-13 09:00:00+00:00,a3d47b1a-985e-5395-a6ee-719dad9b580f,None
2018-06-09 04:00:00+00:00,a3e8e405-0eed-59b6-8747-fe892a6f93de,None
2018-06-13 09:00:00+00:00,a46fb790-028c-5d17-a3e6-7d08daec0c03,None
2018-06-09 04:00:00+00:00,a470ac6d-d448-522b-9b74-8ed27f24de25,None
2018-06-07 00:30:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None
2018-06-07 01:15:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None
2018-06-07 17:30:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None
2018-06-07 18:30:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None
2018-06-07 19:00:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None
2018-06-07 19:30:00+00:00,a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,None


In [40]:
%%sql result_4b <<
CREATE OR REPLACE VIEW complete AS
with missing as (select g_time, id, null as value
                 from gaps, LATERAL generate_series(lagtime + interval '15 minute', 
                            time - interval '15 minutes', interval '15 minutes') as g_time)
SELECT g_time as time, id, CAST(value AS real) FROM missing
UNION ALL
SELECT time, id, value FROM data;

SELECT * FROM complete ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_4b


In [41]:
# Do not delete/edit this cell
result_4b.DataFrame().to_csv('results/result_4b.csv', index=False)

In [42]:
grader.check("q4b")

q4b results: All test cases passed!

### Question 4c: Linear Interpolation
*Note: If you struggled with the previous subparts of this problem, you can use our table `complete_provided` instead of your `complete` table in this subpart.*

Now, given the `complete` view or the `complete_provided` table, the remaining task is to do linear interpolation to fill in the missing values we manufactured in Step 2. We have code from **this semester's** Lecture 14 (Data Cleaning) we can reuse here! In particular, your database already includes the UDA `coalesce_agg` we used in lecture (you can use it directly, there's no need to redefine it).

But note that in Lecture 14's example of linear interpolation we had a field called `entry_rank` that was used to order *all* the records in the table. By contrast, here the ordering we care about is the series of `time` for each sensor `id` *independently*. You will need to make minor changes to adapt the linear interpolation code from class to work here.

Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation *per sensor id over time*. Please **retain all additional columns created from forward and backward passes** (there should be 10 columns in the view - the order of the columns does not matter.) The three cells below correspond to the forward, backward, and final passes from lecture.

In [43]:
%%sql
CREATE OR REPLACE VIEW forward AS
select *, SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END) 
    OVER (partition by id ORDER BY time) AS run,
    coalesce_agg(value) OVER (partition by id ORDER BY time) AS run_start,
    CASE WHEN value IS NULL THEN lead(value, 1) OVER (partition by id ORDER BY time) 
    ELSE NULL
    END AS nextval
    from complete;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.


[]

In [44]:
%%sql
CREATE OR REPLACE VIEW backward AS
select *, CASE WHEN value IS NOT NULL THEN value
            ELSE coalesce_agg(nextval) OVER (PARTITION BY run, id ORDER BY time DESC)
       END AS run_end,
        count(*) OVER (PARTITION BY run, id) AS run_size,
        count(*) OVER (PARTITION BY run, id ORDER BY time) AS run_rank
from forward;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.


[]

In [45]:
%%sql result_4c <<
CREATE OR REPLACE VIEW likely_data AS
SELECT *, run_start + (run_rank-1)*((run_end-run_start)/(run_size)) AS interpolated
  FROM backward;
SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_4c


In [46]:
# Do not delete/edit this cell
result_4c.DataFrame().to_csv('results/result_4c.csv', index=False)

In [47]:
grader.check("q4c")

q4c results: All test cases passed!

## Congratulations! You have finished Project 3.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**For submission on Gradescope, you will need to submit BOTH the proj3.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 3 directory within JupyterHub (remove `proj3.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [48]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1b.csv (deflated 78%)
updating: results/result_1c.csv (deflated 58%)
updating: results/result_2a.csv (deflated 92%)
updating: results/result_2b.csv (deflated 92%)
updating: results/result_3b.csv (deflated 43%)
updating: results/result_3c.csv (stored 0%)
updating: results/result_3e.csv (deflated 76%)
updating: results/result_4a.csv (deflated 73%)
updating: results/result_4b.csv (deflated 91%)
updating: results/result_4c.csv (deflated 86%)


/home/jovyan/fa22/proj/proj3/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [49]:
grader.check_all()

q1b results: All test cases passed!

q1c results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q3e results: All test cases passed!

q4a results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [50]:
# Save your notebook first, then run this cell to export your submission.
grader.export()